
# **LTX2 for Text & Image to Video with ComfyUI**

---

- You can generate up to 3 seconds of a 480p video with the default models and settings on the T4 GPU.
- Run the cell below to get a link (e.g. https://localhost:8188/) which you can use to launch the comfyUI interface.
- Github project page: https://github.com/Lightricks/LTX-2
- Workflows: https://github.com/Isi-dev/Google-Colab_Notebooks/tree/main/ComfyUI/ComfyUI_LTX2
- Models source: (1) https://huggingface.co/Kijai/LTXV2_comfy/tree/main (2) https://huggingface.co/Comfy-Org/ltx-2/tree/main/split_files/text_encoders
- Notebook source: https://github.com/Isi-dev/Google-Colab_Notebooks
- Premium notebooks I highly recommend: https://isinse.gumroad.com/





In [ ]:
# @markdown # 💥Prepare Environment

!pip install torch torchvision

offload_models_for_low_VRAM = True # @param {type:"boolean"}
include_manager = False # @param {type:"boolean"}

%cd /content
from IPython.display import clear_output
clear_output()
!pip install -q torchsde einops diffusers accelerate
!pip install av spandrel albumentations onnx opencv-python onnxruntime
!git clone https://github.com/comfyanonymous/ComfyUI
!pip install -r /content/ComfyUI/requirements.txt
clear_output()


%cd /content/ComfyUI/custom_nodes
if include_manager:
    !git clone --branch com_man_3.37 https://github.com/Isi-dev/ComfyUI-Manager
!git clone https://github.com/kijai/ComfyUI-KJNodes
!git clone https://github.com/city96/ComfyUI-GGUF
%cd /content/ComfyUI/custom_nodes/ComfyUI-KJNodes
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes/ComfyUI-GGUF
!pip install -r requirements.txt
if include_manager:
    %cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
    !pip install -r requirements.txt

clear_output()


%cd /content/ComfyUI


import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import subprocess
import sys
from pathlib import Path
# sys.path.insert(0, '/content/ComfyUI')

def install_apt_packages():
    packages = ['aria2']

    try:
        # Run apt install silently (using -qq)
        subprocess.run(
            ['apt-get', '-y', 'install', '-qq'] + packages,
            check=True,
            capture_output=True
        )
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")


print("Installing apt packages...")
install_apt_packages()

def download_with_aria2c(link, folder="/content/ComfyUI/models/loras"):
    import os

    filename = link.split("/")[-1]
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d {folder} -o {filename}"

    print("Executing download command:")
    print(command)

    os.makedirs(folder, exist_ok=True)
    get_ipython().system(command)

    return filename



def download_civitai_model(civitai_link, civitai_token, folder="/content/ComfyUI/models/loras"):
    import os
    import time

    os.makedirs(folder, exist_ok=True)

    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        raise ValueError("Invalid Civitai URL format. Please use a link like: https://civitai.com/api/download/models/1523247?...")

    civitai_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if civitai_token:
        civitai_url += f"&token={civitai_token}"

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"model_{timestamp}.safetensors"

    full_path = os.path.join(folder, filename)

    download_command = f"wget --max-redirect=10 --show-progress \"{civitai_url}\" -O \"{full_path}\""
    print("Downloading from Civitai...")

    os.system(download_command)

    local_path = os.path.join(folder, filename)
    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"LoRA downloaded successfully: {local_path}")
    else:
        print(f"❌ LoRA download failed or file is empty: {local_path}")

    return filename

def download_lora(link, folder="/content/ComfyUI/models/loras", civitai_token=None):
    """
    Download a model file, automatically detecting if it's a Civitai link or huggingface download.

    Args:
        link: The download URL (either huggingface or Civitai)
        folder: Destination folder for the download
        civitai_token: Optional token for Civitai downloads (required if link is from Civitai)

    Returns:
        The filename of the downloaded model
    """
    if "civitai.com" in link.lower():
        if not civitai_token:
            raise ValueError("Civitai token is required for Civitai downloads")
        return download_civitai_model(link, civitai_token, folder)
    else:
        return download_with_aria2c(link, folder)

def model_download(url: str, dest_dir: str, filename: str = None, silent: bool = True) -> bool:
    """
    Colab-optimized download with aria2c

    Args:
        url: Download URL
        dest_dir: Target directory (will be created if needed)
        filename: Optional output filename (defaults to URL filename)
        silent: If True, suppresses all output (except errors)

    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Create destination directory
        Path(dest_dir).mkdir(parents=True, exist_ok=True)

        # Set filename if not specified
        if filename is None:
            filename = url.split('/')[-1].split('?')[0]  # Remove URL parameters

        # Build command
        cmd = [
            'aria2c',
            '--console-log-level=error',
            '-c', '-x', '16', '-s', '16', '-k', '1M',
            '-d', dest_dir,
            '-o', filename,
            url
        ]

        # Add silent flags if requested
        if silent:
            cmd.extend(['--summary-interval=0', '--quiet'])
            print(f"Downloading {filename}...", end=' ', flush=True)

        # Run download
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)

        if silent:
            print("Done!")
        else:
            print(f"Downloaded {filename} to {dest_dir}")
        return filename

    except subprocess.CalledProcessError as e:
        error = e.stderr.strip() or "Unknown error"
        print(f"\nError downloading {filename}: {error}")
        return False
    except Exception as e:
        print(f"\nError: {str(e)}")
        return False


ltx_model = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/diffusion_models/ltx-2-19b-distilled_Q4_K_M.gguf" # @param {"type":"string"}
dit_model=model_download(ltx_model, "/content/ComfyUI/models/unet")

text_encoder_link = "https://huggingface.co/Comfy-Org/ltx-2/resolve/main/split_files/text_encoders/gemma_3_12B_it_fp4_mixed.safetensors" # @param {"type":"string"}
text_encoder_model = model_download(text_encoder_link, "/content/ComfyUI/models/text_encoders")

text_encoder2_link = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/text_encoders/ltx-2-19b-embeddings_connector_distill_bf16.safetensors" # @param {"type":"string"}
text_encoder2_model = model_download(text_encoder2_link, "/content/ComfyUI/models/text_encoders")

vae_link = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/VAE/LTX2_video_vae_bf16.safetensors" # @param {"type":"string"}
vae_model = model_download(vae_link, "/content/ComfyUI/models/vae")

vae_audio_link = "https://huggingface.co/Kijai/LTXV2_comfy/resolve/main/VAE/LTX2_audio_vae_bf16.safetensors" # @param {"type":"string"}
vae_audio_model = model_download(vae_audio_link, "/content/ComfyUI/models/vae")

upscaler_link = "https://huggingface.co/Lightricks/LTX-2/resolve/main/ltx-2-spatial-upscaler-x2-1.0.safetensors" # @param {"type":"string"}
upscaler_model = model_download(upscaler_link, "/content/ComfyUI/models/latent_upscale_models")






download_loRA1 = True # @param {type:"boolean"}
lora1_download_url = "https://huggingface.co/Lightricks/LTX-2-19b-LoRA-Camera-Control-Dolly-Left/resolve/main/ltx-2-19b-lora-camera-control-dolly-left.safetensors"# @param {"type":"string"}

download_loRA_2 = False # @param {type:"boolean"}
lora_2_download_url = "Put your loRA here"# @param {"type":"string"}

download_loRA_3 = False # @param {type:"boolean"}
lora3_download_url = "Put your loRA here"# @param {"type":"string"}

download_loRA_4 = False # @param {type:"boolean"}
lora_4_download_url = "Put your loRA here"# @param {"type":"string"}

token_if_civitai_url = ""# @param {"type":"string"}



lora1 = None
if download_loRA1:
    lora1 = download_lora(lora1_download_url, civitai_token=token_if_civitai_url)
# Validate loRA file extension
valid_extensions = {'.safetensors', '.ckpt', '.pt', '.pth', '.sft'}
if lora1:
    if not any(lora1.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora1}")
        lora1 = None
    else:
        clear_output()
        print("loRA 1 downloaded succesfully!")

lora_2 = None
if download_loRA_2:
    lora_2 = download_lora(lora_2_download_url, civitai_token=token_if_civitai_url)
if lora_2:
    if not any(lora_2.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_2}")
        lora_2 = None
    else:
        clear_output()
        print("loRA 2 downloaded succesfully!")

lora_3 = None
if download_loRA_3:
    lora_3 = download_lora(lora3_download_url, civitai_token=token_if_civitai_url)
if lora_3:
    if not any(lora_3.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_3}")
        lora_3 = None
    else:
        clear_output()
        print("loRA 3 downloaded succesfully!")

lora_4 = None
if download_loRA_4:
    lora_4 = download_lora(lora_4_download_url, civitai_token=token_if_civitai_url)
if lora_4:
    if not any(lora_4.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora_4}")
        lora_4 = None
    else:
        clear_output()
        print("loRA 4 downloaded succesfully!")



use_cloudflare = False  # @param {type:"boolean"}
use_interface_in_cell = False  # @param {type:"boolean"}
use_ngrok = False  # @param {type:"boolean"}
NGROK_AUTH_TOKEN = ""  # @param {type:"string"}

# ---------------- NGROK IMPLEMENTATION ----------------
if use_ngrok:
    print("⏳ Setting up pyngrok tunnel...")

    # Install pyngrok
    !pip install -q pyngrok

    import time, socket, threading
    from pyngrok import ngrok, conf
    from IPython.display import clear_output

    # Apply the ngrok authtoken (if provided)
    if NGROK_AUTH_TOKEN.strip():
        print("🔐 Using provided ngrok authtoken...")
        conf.get_default().auth_token = NGROK_AUTH_TOKEN
    else:
        print("⚠️ No authtoken provided. Please stop this execution and input your Authtoken.")

    def launch_ngrok(port):
        # Wait for ComfyUI server to be ready before tunneling
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(("127.0.0.1", port))
            sock.close()
            if result == 0:
                break

        print("\n🚀 ComfyUI loaded. Launching ngrok tunnel...\n")

        # Close any previous tunnels just in case
        ngrok.kill()

        # Create the HTTP tunnel
        public_url = ngrok.connect(addr=f"http://127.0.0.1:{port}", proto="http")
        clear_output()
        print("✅ ComfyUI is ready!")
        print(f"🌐 Access it here: {public_url.public_url}")

    threading.Thread(target=launch_ngrok, daemon=True, args=(8188,)).start()

    if offload_models_for_low_VRAM:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server


# ---------------- CLOUDFLARE IMPLEMENTATION ----------------
elif use_cloudflare:
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1

    import subprocess, threading, time, socket
    from IPython.display import clear_output

    def cloudflare_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        print("\n🚀 ComfyUI loaded. Launching Cloudflare tunnel...\n")

        p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in p.stderr:
            l = line.decode()
            if "trycloudflare.com " in l:
                clear_output()
                print("✅ ComfyUI is ready!")
                print("🌐 Access it here:", l[l.find("http"):], end='')
                break

    threading.Thread(target=cloudflare_thread, daemon=True, args=(8188,)).start()

    if offload_models_for_low_VRAM:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

# ---------------- IN-CELL INTERFACE ----------------
elif use_interface_in_cell:
    import threading, time, socket
    from google.colab import output
    from IPython.display import clear_output

    def iframe_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        output.serve_kernel_port_as_iframe(port, height=1024)
        clear_output()
        print("✅ ComfyUI loaded inside the notebook.")
        print("To open it in a separate tab, click the link below:")
        output.serve_kernel_port_as_window(port)

    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    if offload_models_for_low_VRAM:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server

# ---------------- DEFAULT (COLAB NATIVE PORT FORWARDING) ----------------
else:
    import socket, time, threading
    from google.colab import output
    from IPython.display import clear_output

    def colab_link_thread(port):
        while True:
            time.sleep(0.5)
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                break
            sock.close()
        clear_output()
        print("✅ ComfyUI is ready!")
        print("Click below to open the interface:")
        output.serve_kernel_port_as_window(port)

    threading.Thread(target=colab_link_thread, daemon=True, args=(8188,)).start()

    if offload_models_for_low_VRAM:
        !python main.py --cache-none --dont-print-server
    else:
        !python main.py --dont-print-server





